420-A52-SF - Algorithmes d'apprentissage supervisé - Hiver 2020 - Spécialisation technique en Intelligence Artificielle<br/>
MIT License - Copyright (c) 2020 Mikaël Swawola
<br/>
![Travaux Pratiques - Validation croisée](static/12-tp-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif la mise en oeuvre de la validation croisée à k plis à l'aide de la librairie **scikit-learn**. Les modèles et algorithmes utilisés seront la régression logistique et la classification KNN Le jeu de données sera de nouveau **Heart**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 0 - Chargement des bibliothèques

In [ ]:
# Manipulation de données
import numpy as np
import pandas as pd

# Visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns

# Helpers
from helpers import polynomial

# Outils divers
from tqdm import tqdm
from collections import defaultdict

# Machine Learning
# Compléter au fur et à mesure du TP l'importation des modules scikit-learn requis

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [ ]:
# Configuration de la visualisation
sns.set(style="darkgrid", rc={'figure.figsize':(12,6)})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

## 1 - Lecture du jeu de données

<strong style="color: #4a86e8">Exercice 1-1: lire le fichier `Heart.csv`<strong/>

In [ ]:
# Compléter le code ci-dessous ~ 1-2 lignes

cols = ['Age','Sex','ChestPain','RestBP','Chol','Fbs','RestECG','MaxHR','ExAng','Oldpeak','Slope','Ca','Thal','AHD']
HRT = pd.read_csv('../../data/Heart.csv', usecols=cols)

In [ ]:
# On supprime tout de suite les données manquantes. Ceci sera vu plus en détail plus tard dans le cours
HRT = HRT.dropna()

<strong style="color: #4a86e8">Exercice 1-2: afficher les dix premières lignes de la trame de données HRT</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

HRT.head()

## 2 - Préparation de données

Nous allons considérer l'ensemble des variables explicatives du jeu de données **Heart**

<strong style="color: #4a86e8">Exercice 2-1: Encoder les variables explicatives catégorielles. Utiliser le numpy array `X` pour stocker les variables explicatives et le vecteur `y` pour la variable réponse. Indice: utilisez pandas ;-)</strong>

In [ ]:
# Compléter le code ci-dessous ~ 2-3 lignes

HRT_onehot = pd.get_dummies(HRT, columns=['ChestPain','Thal'], prefix = ['cp','thal'], drop_first=True)
X = HRT_onehot.drop(['AHD'], axis=1).values
y = (HRT['AHD'] == "Yes").astype(int).values

<strong style="color: #4a86e8">Exercice 2-1: Quel est le nombre de variables explicatives contenues dans `X` ?</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X.shape[1]

<strong style="color: #4a86e8">Exercice 2-2: Quel est le nombre d'observations ?</strong>

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X.shape[0]

## 3 - Validation croisée à 5 plis - Régression logistique

### 3 - 1 - Mélange des données et séparation en jeu d’entraînement et de test

<strong style="color: #4a86e8">Exercice 3-1: à l'aide de scikit-learn, sépararer les données en jeu d'entraînement et jeu de test. La taille du jeu de test doit représenter 30% de la taille du jeu de données et l'état du générateur aléatoire sera fixé à 2020 afin de permettre la reproductibilité</strong>

![3 - 1 - Mélange des données et séparation en jeu d’entraînement et de test](static/fig1.png)

In [ ]:
# Compléter le code ci-dessous ~ 1 ligne

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2020)

Vérification des dimensions de `X_train` et `X_test`

In [ ]:
print(X_train.shape)
print(X_test.shape)

### 3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur 5 plis de validation

<strong style="color: #4a86e8">Exercice 3-2: réaliser une validation croisée à 5-plis sur une régression logistique polynomiale (sans variables d'interaction) en faisant varier l'ordre `n` du polynôme de 1 à 5. Vous pouvez utiliser la fonction `polynomial` disponible dans `helpers.py`. Enregistrez dans le dictionnaire `history` les scores (accuracy) sur les jeux d'entraînement et de validation pour chaque valeurs de `n`</strong><br/><br/>

<strong style="color: green">Afin de faciliter l'écriture du code, cet exercice sera réalisé en deux étapes
<ul>
    <li>Etape 1: effectuer une validation croisée à 5 plis sur une régression logistique d'ordre 1 seulement</li>
    <li>Etape 2: inclure le code précédent dans une boucle for en faisant varier l'ordre de 1 à 5</li>
</ul>
<strong>

Instanciation de `scaler` pour la standardisation des données

In [ ]:
scaler = StandardScaler()

### Etape 1 : effectuer une validation croisée à 5 plis sur une régression logistique d'ordre 1 seulement

![3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur k plis de validation](static/fig2.png)

[sklearn.model_selection.KFold(n_splits=5, shuffle=False, random_state=None)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

In [ ]:
# Compléter le code ci-dessous ~ 10-20 lignes ...

# Pseudo code:
# ------------
#
# Instancier KFold (sklearn)
#
# Ajouter x0 sur X
#
# Standardiser X
#
# Itérer sur les k plis:
#     Effectuer la classification sur le plis courant (jeu d'entraînement)
#     Évaluer les performances sur le jeu de validation

history = defaultdict(list)

kf = KFold(n_splits=5)

# Variables polynomiales
X_temp = polynomial(X_train, degree=1)

# Standardisation
scaler.fit(X_temp)
X_train_scaled = scaler.transform(X_temp)

for train_index, val_index in kf.split(X_train_scaled):
    # Préparation des plis
    x_cv_train, x_cv_val = X_train_scaled[train_index,:], X_train_scaled[val_index,:]
    y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

    # Régression logistique
    clf = LogisticRegression(penalty="none", fit_intercept=False).fit(x_cv_train, y_cv_train)
    
    # Record performances par plis
    history['train'].append(clf.score(x_cv_train, y_cv_train))
    history['val'].append(clf.score(x_cv_val, y_cv_val))
    
print(f"Accuracy de validation = {np.mean(history['val'])}")
print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

### Etape 2 - inclure le code précédent dans une boucle for en faisant varier l'ordre de 1 à 5

![3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur k plis de validation](static/fig3.png)

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

def LogisticRegressionCV(X_train_scaled, y_train, n_splits=5):
    history = defaultdict(list)
    
    kf = KFold(n_splits)

    for train_index, val_index in kf.split(X_train_scaled):
        # Préparation des plis
        x_cv_train, x_cv_val = X_train_scaled[train_index,:], X_train_scaled[val_index,:]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

        # Régression logistique
        clf = LogisticRegression(penalty="none", solver="sag", max_iter=1000000, fit_intercept=False).fit(x_cv_train, y_cv_train)

        # Record performances par plis
        history['train'].append(clf.score(x_cv_train, y_cv_train))
        history['val'].append(clf.score(x_cv_val, y_cv_val))
        
    return np.mean(history['train']), np.mean(history['val'])


history = defaultdict(list)
for n in range(1, 10):
    
    # variables polynomiales
    X_temp = polynomial(X_train, degree=n)

    # Standardisation
    scaler.fit(X_temp)
    X_train_scaled = scaler.transform(X_temp)
    
    tr, val = LogisticRegressionCV(X_train_scaled, y_train)
    history['train'].append(tr)
    history['val'].append(val)

### 3 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation

![3 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation](static/fig4.png)

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1,10), history['train'], label="train")
ax.plot(range(1,10), history['val'], label="test")
ax.set_xlabel('n', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 3 - 4 - Entrainement du meilleur modèle

<strong style="color: #4a86e8">Exercice 3-4 : utiliser les résultas précédents pour sélectionner le meilleur modèle, et réentrainer ce modèle sur l'ensemble des données d'entraînement</strong>

![3 - 4 - Entrainement du meilleur modèle](static/fig5.png)

In [ ]:
# Compléter le code ci-dessous ~ quelques lignes ...

# Variables polynomiales
X_train_temp = polynomial(X_train, degree=2)

# Standardisation
scaler.fit(X_train_temp)
X_train_scaled = scaler.transform(X_train_temp)

clf_best = LogisticRegression(penalty="none", max_iter=100000, fit_intercept=False).fit(X_train_scaled, y_train)

### 3 - 5 - Performances du meilleur modèle

<strong style="color: #4a86e8">Exercice 3-5 : Évaluer les performances du meilleur modèle sur le jeu de test</strong>
![3 - 5 - Performances du modèle final](static/fig6.png)

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

# Variables polynomiales
X_test_temp = polynomial(X_test, degree=2)

# Standardisation
scaler.fit(X_test_temp)
X_test_scaled = scaler.transform(X_test_temp)

In [ ]:
clf_best.score(X_test_scaled, y_test)

## 4 - Validation croisée à 5 plis - Algorithme des k plus proches voisins

### 4 - 1 - Mélange des données et séparation en jeu d’entraînement et de test

Vous pouvez reprendre `X_train`, `X_test`, `y_train` et `y_test` obtenus au **3-1**

### 4 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur 5 plis de validation

<strong style="color: #4a86e8">Exercice 4-2: réaliser une validation croisée à k-plis sur une classification KNN en faisant varier le nombre de voisins `k` de 1 à 20. Enregistrez les scores (accuracy) sur les jeux d'entraînement et de validation pour chaque valeur de `k` dans le dictionnaire `history`</strong><br/><br/>
<strong style="color: green">Comme pour l'exercice 3, afin de faciliter l'écriture du code, cette question sera faite en deux étapes
<ul>
    <li>Etape 1: effectuer une validation croisée à 5 plis sur une classification KNN avec k=10 voisins seulement</li>
    <li>Etape 2: inclure le code précédent dans une boucle for en faisant varier le nombre de voisins de 1 à 20</li>
</ul>
<strong>

### Etape 1 - effectuer une validation croisée à 5 plis sur une classification KNN avec k=10 voisins seulement

In [ ]:
# Compléter le code ci-dessous ~ 10-15 lignes ...

history = defaultdict(list)

kf = KFold(n_splits=5)

k = 10

for train_index, val_index in kf.split(X_train):
    
    x_cv_train, x_cv_val = X_train_scaled[train_index,:], X_train_scaled[val_index,:]
    y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]
    
    # Classification KNN
    neigh = KNeighborsClassifier(n_neighbors=k).fit(x_cv_train, y_cv_train)
    
    # Record performances par plis
    history['train'].append(neigh.score(x_cv_train, y_cv_train))
    history['val'].append(neigh.score(x_cv_val, y_cv_val))

print(f"Accuracy de validation = {np.mean(history['val'])}")
print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

### Etape 2 -  inclure le code précédent dans une boucle for en faisant varier le nombre de voisins de 1 à 20

In [ ]:
# Compléter le code ci-dessous ~ quelques lignes ...

def KNeighborsClassifierCV(k):
    history = defaultdict(list)
    
    kf = KFold(n_splits=5)

    for train_index, val_index in kf.split(X_train):
        x_cv_train, x_cv_val = X_train[train_index,:], X_train[val_index,:]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

        # Classification KNN
        neigh = KNeighborsClassifier(n_neighbors=k).fit(x_cv_train, y_cv_train)
    
        # Record performances par plis
        history['train'].append(neigh.score(x_cv_train, y_cv_train))
        history['val'].append(neigh.score(x_cv_val, y_cv_val))
        
    return np.mean(history['train']), np.mean(history['val'])

    
history = defaultdict(list)
for k in range(1, 20):
    t, v = KNeighborsClassifierCV(k)
    history['train'].append(t)
    history['val'].append(v)

### 4 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1, 20), history['train'], label="train")
ax.plot(range(1, 20), history['val'], label="test")
ax.set_xlabel('k', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 4 - 4 - Entrainement du meilleur modèle

<strong style="color: #4a86e8">Exercice 4-4: utiliser les résultas précédents pour sélectionner le meilleur modèle, et réentrainer ce modèle sur l'ensemble des données d'entraînement</strong>

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

neigh_best = KNeighborsClassifier(n_neighbors=15).fit(X_train, y_train)

### 4 - 5 - Performances du meilleur modèle

<strong style="color: #4a86e8">Exercice 4-5: Évaluer les performances du meilleur modèle sur le jeu de test</strong>

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

neigh_best.score(X_test, y_test)

<strong style="color: #4a86e8">Exercice 4-6: Expliquer ces performances</strong>

### 5 - Choix du modèle final

<strong style="color: #4a86e8">Exercice 5: Quel modèle choisissez-vous ? Quels sont les valeurs des hyperparamètres ?</strong>

### Fin du TP